# Topic Modeling - academia
- title, abstract만 갖고 토픽 모델링 진행
## 전처리
- NER로 사람 이름 체크 및 제거
- 불용어로 Information System, Research, Study 제거
- 각 키워드를 소문자 변환 후에 띄어쓰기 제거 및 복수형태 통일
## 모델
Num Topics =  1 → Coherence = 0.2463
Num Topics =  2 → Coherence = 0.3622
Num Topics =  3 → Coherence = 0.3344
Num Topics =  4 → Coherence = 0.3213
Num Topics =  5 → Coherence = 0.3441
Num Topics =  6 → Coherence = 0.3295
Num Topics =  7 → Coherence = 0.3725
Num Topics =  8 → Coherence = 0.3573
Num Topics =  9 → Coherence = 0.3487
Num Topics = 10 → Coherence = 0.3393
Num Topics = 11 → Coherence = 0.3501
Num Topics = 12 → Coherence = 0.3446
Num Topics = 13 → Coherence = 0.3465
Num Topics = 14 → Coherence = 0.3394
Num Topics = 15 → Coherence = 0.3569
Num Topics = 16 → Coherence = 0.3518
Num Topics = 17 → Coherence = 0.3518
Num Topics = 18 → Coherence = 0.3420
Num Topics = 19 → Coherence = 0.3515
Num Topics = 20 → Coherence = 0.3407

Optimal #Topics = 7 with coherence 0.3725

## 결과

=== Top 7 Topics ===
Topic  0: system, data, model, learning, based, approach, paper, design, process, analysis
Topic  1: worker, market, power, crisis, platform, work, impact, service, demand, job
Topic  2: user, work, interaction, effect, identity, agent, perception, technology, influence, result
Topic  3: platform, effect, medium, consumer, product, user, online, information, impact, content
Topic  4: organization, technology, innovation, student, use, case, strategy, development, process, business
Topic  5: review, design, knowledge, platform, ecosystem, project, literature, information, value, business
Topic  6: data, model, privacy, health, based, information, using, risk, method, result

Coherence (c_v) = 0.3725



In [15]:
import pandas as pd

acad=pd.read_csv('data/03_journal_2023_2025_with_methods.csv')
acad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3209 entries, 0 to 3208
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         3209 non-null   object
 1   date          3209 non-null   int64 
 2   abstract      3209 non-null   object
 3   keywords      3209 non-null   object
 4   authors       3209 non-null   object
 5   affiliations  3209 non-null   object
 6   methods       3209 non-null   object
dtypes: int64(1), object(6)
memory usage: 175.6+ KB


In [16]:
acad.head()

,title,date,abstract,keywords,authors,affiliations,methods
0,Digital “x”?Charting a Path for Digital-Themed...,2023,"As of late, the use of “digital” as a qualifie...","digital x, IT x, digitalization, digitization,...","Abayomi Baiyere, Varun Grover, Kalle J. Lyytin...",ISR,The methodology of this paper is qualitative. ...
1,"Law, Economics, and Privacy: Implications of G...",2023,Widespread abuse of internet users’ privacy on...,"data protection regulation, government policy,...","Ram D. Gopal, Hooman Hidaji, Sule Nur Kutlu, R...",ISR,The paper uses a stylized analytical model to ...
2,Spoiled for Choice? Personalized Recommendatio...,2023,Online healthcare platforms provide users with...,"personal health management, online healthcare ...","Tongxin Zhou, Yingfei Wang, Lu (Lucy) Yan, Yon...",ISR,We propose a personalized recommendation frame...
3,A Theory-Driven Deep Learning Method for Voice...,2023,As artificial intelligence and digitalization ...,"customer response prediction, voice chat, theo...","Gang Chen, Shuaiyong Xiao, Chenghong Zhang, Hu...",ISR,We propose a theory-driven deep learning metho...
4,The Decoy Effect and Recommendation Systems,2023,"In this paper, we explore the decoy effect in ...","recommendation system, personalization, decoy ...","Nasim Mousavi, Panagiotis Adamopoulos, Jesse B...",ISR,The methodology of this paper is to conduct a ...


In [17]:
# !pip install spacy nltk gensim
# !python -m spacy download en_core_web_sm

import re
import pandas as pd
import spacy
import nltk

from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel

# ——— NLTK 리소스 다운로드 ———
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# ——— spaCy 로드 (NER 용) ———
nlp = spacy.load("en_core_web_sm")

# ——— 사용자 정의 불용어 ———
CUSTOM_STOPWORDS = {'information system', 'research', 'study'}


# ——— NLTK 리소스 다운로드 ———
nltk.download('punkt')
nltk.download('wordnet')

# ——— spaCy 모델 로드 ———
nlp = spacy.load("en_core_web_sm")
lemmatizer = WordNetLemmatizer()

# ——— 사용자 정의 불용어 (소문자/공백제거 후 기준) ———
CUSTOM_STOPWORDS = {'information system','information','system', 'research', 'study'}


def preprocess_text(text: str) -> list[str]:
    """
    1) NaN 또는 리스트 처리
    2) HTML 태그 및 수식 제거
    3) 알파벳·공백 외 문자 제거 → 소문자화
    4) 토큰화 → NLTK 불용어 & 길이 >2 필터
    5) POS 태그 필터링 (명사·동사 계열)
    6) Lemmatize
    7) spaCy NER로 PERSON 엔터티 제거
    8) 사용자 정의 불용어 제외
    """
    # 1) Null / 리스트 처리
    if pd.isnull(text):
        return []
    if isinstance(text, list):
        text = ' '.join(text)

    # 2) HTML, LaTeX 수식 제거
    if '<' in text and '>' in text:
        text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r"\$.*?\$", "", text)
    text = re.sub(r"\\\(.*?\\\)", "", text)

    # 3) 알파벳·공백 외 제거 → 소문자
    text = re.sub(r"[^a-zA-Z\s]", " ", text).lower()

    # 4) 토큰화 → 불용어·길이 필터
    stop_words = set(stopwords.words('english'))
    tokens = [
        w for w in word_tokenize(text)
        if w not in stop_words and len(w) > 2
    ]

    # 5) POS 태깅 → 명사·동사 계열만
    allowed = {'NN','NNS','NNP','NNPS','VB','VBD','VBG','VBN','VBP','VBZ'}
    tokens = [w for w, pos in pos_tag(tokens) if pos in allowed]

    # 6) 표제어 추출
    lemm = WordNetLemmatizer()
    tokens = [lemm.lemmatize(w) for w in tokens]

    # 7) spaCy NER 적용 → PERSON 제거
    doc = nlp(" ".join(tokens))
    tokens = [tok.text for tok in doc if tok.ent_type_ != 'PERSON']

    # 8) 사용자 정의 불용어 제거
    return [w for w in tokens if w not in CUSTOM_STOPWORDS]


# ——— 데이터 불러오기 ———
# df = pd.read_csv('your_data.csv')  # title, abstract 컬럼 포함
# 예시: title + abstract 합치기
acad['text'] = acad['title'].fillna('') + ' ' + acad['abstract'].fillna('')+acad['keywords'].fillna('')

# ——— 전처리 적용 ———
texts = [preprocess_text(doc) for doc in acad['text']]

# ——— Gensim Dictionary & Corpus 생성 ———
dictionary = corpora.Dictionary(texts)
# (선택) 너무 드문 단어/전체의 절반 이상 등장 단어 필터링
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(txt) for txt in texts]

# ——— 최적 토픽 수 탐색 (1~20) ———
coherence_scores = []
for k in range(1, 21):
    lda = LdaModel(corpus=corpus,
                   id2word=dictionary,
                   num_topics=k,
                   random_state=42,
                   passes=10)
    cm = CoherenceModel(model=lda,
                        texts=texts,
                        dictionary=dictionary,
                        coherence='c_v')
    score = cm.get_coherence()
    coherence_scores.append((k, score))
    print(f"Num Topics = {k:2d} → Coherence = {score:.4f}")

# ——— 최적 토픽 수 출력 ———
best_k, best_score = max(coherence_scores, key=lambda x: x[1])
print(f"\nOptimal #Topics = {best_k} with coherence {best_score:.4f}")

[nltk_data] Downloading package punkt to /Users/choihj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/choihj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/choihj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/choihj/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/choihj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/choihj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Num Topics =  1 → Coherence = 0.2144
Num Topics =  2 → Coherence = 0.3410
Num Topics =  3 → Coherence = 0.3175
Num Topics =  4 → Coherence = 0.3187
Num Topics =  5 → Coherence = 0.3260
Num Topics =  6 → Coherence = 0.3173
Num Topics =  7 → Coherence = 0.3025
Num Topics =  8 → Coherence = 0.3564
Num Topics =  9 → Coherence = 0.3417
Num Topics = 10 → Coherence = 0.3439
Num Topics = 11 → Coherence = 0.3397
Num Topics = 12 → Coherence = 0.3269
Num Topics = 13 → Coherence = 0.3446
Num Topics = 14 → Coherence = 0.3545
Num Topics = 15 → Coherence = 0.3433
Num Topics = 16 → Coherence = 0.3414
Num Topics = 17 → Coherence = 0.3505
Num Topics = 18 → Coherence = 0.3413
Num Topics = 19 → Coherence = 0.3349
Num Topics = 20 → Coherence = 0.3513

Optimal #Topics = 8 with coherence 0.3564


In [18]:
# pip install spacy nltk gensim beautifulsoup4
# python -m spacy download en_core_web_sm

import re
import pandas as pd
import spacy
import nltk

from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel

# ——— NLTK 리소스 다운로드 ———
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# ——— spaCy 로드 (NER 용) ———
nlp = spacy.load("en_core_web_sm")

CUSTOM_STOPWORDS = {'information system','information','system', 'research', 'study'}

def preprocess_text(text: str) -> list[str]:
    """
    1) NaN 또는 리스트 처리
    2) HTML 태그 및 수식 제거
    3) 알파벳·공백 외 문자 제거 → 소문자화
    4) 토큰화 → NLTK 불용어 & 길이 >2 필터
    5) POS 태그 필터링 (명사·동사 계열)
    6) Lemmatize
    7) spaCy NER로 PERSON 엔터티 제거
    8) 사용자 정의 불용어 제외
    """
    # 1) Null / 리스트 처리
    if pd.isnull(text):
        return []
    if isinstance(text, list):
        text = ' '.join(text)

    # 2) HTML, LaTeX 수식 제거
    if '<' in text and '>' in text:
        text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r"\$.*?\$", "", text)
    text = re.sub(r"\\\(.*?\\\)", "", text)

    # 3) 알파벳·공백 외 제거 → 소문자
    text = re.sub(r"[^a-zA-Z\s]", " ", text).lower()

    # 4) 토큰화 → 불용어·길이 필터
    stop_words = set(stopwords.words('english'))
    tokens = [
        w for w in word_tokenize(text)
        if w not in stop_words and len(w) > 2
    ]

    # 5) POS 태깅 → 명사·동사 계열만
    allowed = {'NN','NNS','NNP','NNPS','VB','VBD','VBG','VBN','VBP','VBZ'}
    tokens = [w for w, pos in pos_tag(tokens) if pos in allowed]

    # 6) 표제어 추출
    lemm = WordNetLemmatizer()
    tokens = [lemm.lemmatize(w) for w in tokens]

    # 7) spaCy NER 적용 → PERSON 제거
    doc = nlp(" ".join(tokens))
    tokens = [tok.text for tok in doc if tok.ent_type_ != 'PERSON']

    # 8) 사용자 정의 불용어 제거
    return [w for w in tokens if w not in CUSTOM_STOPWORDS]


def run_lda_pipeline(
    df: pd.DataFrame,
    text_cols: list[str] = ['title', 'abstract','keywords'],
    num_topics: int = 8, # 토픽 개수 설정해야함
    no_below: int = 5,
    no_above: float = 0.5,
    passes: int = 10
) -> tuple[LdaModel, corpora.Dictionary, list, float]:
    """
    1) df[text_cols] 합치고 preprocess_text 적용
    2) Dictionary & Corpus 생성 (filter_extremes)
    3) LDA 학습(num_topics)
    4) 토픽 키워드 출력 & coherence 계산
    Returns: (lda_model, dictionary, corpus, coherence_score)
    """
    df = df.copy()
    df['__text'] = df[text_cols].fillna('').agg(' '.join, axis=1)
    texts = [preprocess_text(doc) for doc in df['__text']]

    dictionary = corpora.Dictionary(texts)
    dictionary.filter_extremes(no_below=no_below, no_above=no_above)
    corpus = [dictionary.doc2bow(txt) for txt in texts]

    lda = LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=num_topics,
        random_state=42,
        passes=passes
    )

    print(f"\n=== Top {num_topics} Topics ===")
    for tid, terms in lda.show_topics(num_topics=num_topics, num_words=10, formatted=False):
        print(f"Topic {tid:2d}: {', '.join([t for t,_ in terms])}")

    cm = CoherenceModel(model=lda, texts=texts, dictionary=dictionary, coherence='c_v')
    coh = cm.get_coherence()
    print(f"\nCoherence (c_v) = {coh:.4f}")

    return lda, dictionary, corpus, coh

[nltk_data] Downloading package punkt to /Users/choihj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/choihj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/choihj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/choihj/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [19]:
lda_model, dictionary, corpus, coherence = run_lda_pipeline(acad)


=== Top 8 Topics ===
Topic  0: data, model, learning, based, process, approach, business, method, decision, machine
Topic  1: customer, review, service, consumer, product, effect, decision, impact, agent, city
Topic  2: data, privacy, security, cybersecurity, network, risk, using, image, power, cyber
Topic  3: work, technology, innovation, student, game, role, development, organization, finding, team
Topic  4: platform, market, effect, sharing, network, consumer, product, impact, user, strategy
Topic  5: firm, performance, effect, market, project, innovation, success, data, business, level
Topic  6: design, technology, health, service, organization, use, healthcare, management, case, process
Topic  7: medium, model, user, language, community, llm, analysis, using, result, content

Coherence (c_v) = 0.3564


In [21]:
print(acad.value_counts(subset='date'))

date
2024    1275
2023    1210
2025     724
Name: count, dtype: int64


# 모델 저장 및 distribution 저장

In [33]:
import pandas as pd
from gensim import corpora, models

# 1. LDA 모델 저장
# 학습이 끝난 lda_model 객체가 있다고 가정합니다.
lda_model.save('acadmodel/lda_model.model')

# 2. 문서-토픽 분포 추출
# corpus: LDA에 입력했던 Bow/BOW-TFIDF 형태의 말뭉치 리스트
# min_probability=0 으로 설정하면, 확률이 0인 토픽도 반환해줍니다.
doc_topics = [
    lda_model.get_document_topics(doc, minimum_probability=0)
    for doc in corpus
]

# 3. 분포를 DataFrame으로 변환
# 각 문서별로 토픽 확률만 추출 → 리스트 of lists
distribution = [
    [prob for _, prob in topic_dist]
    for topic_dist in doc_topics
]

# 컬럼명 생성 (예: 'Topic_0', 'Topic_1', ...)
num_topics = lda_model.num_topics
columns = [f'Topic_{i}' for i in range(num_topics)]

df_dist = pd.DataFrame(distribution, columns=columns)

# 4. CSV로 저장
df_dist.to_csv('data/03_academia_lda_document_topic_distribution.csv', index=False)

print("LDA 모델과 문서-토픽 분포 CSV 저장이 완료되었습니다.")

LDA 모델과 문서-토픽 분포 CSV 저장이 완료되었습니다.


In [34]:
import pandas as pd
from gensim import corpora, models

# —————————————————————————————————————
# (이전 코드로 얻은) document–topic distribution DataFrame: df_dist
# 컬럼명은 'Topic_0', 'Topic_1', … 'Topic_{num_topics-1}'
# —————————————————————————————————————

# 1) 제외할 토픽 번호 리스트 지정
# 예시: 토픽 1, 9, 17을 제외하고 싶으면
topics_to_exclude = []

# 2) 제외할 토픽을 뺀 컬럼 리스트 생성
remaining_cols = [
    col for col in df_dist.columns
    if int(col.split('_')[1]) not in topics_to_exclude
]

# 3) 제외된 컬럼들 중 가장 확률이 높은 토픽 이름(예: 'Topic_5')을 찾아서,
#    숫자 부분만 정수로 추출해 'dominant_topic_excl' 컬럼에 저장
df_dist['dominant_topic_excl'] = (
    df_dist[remaining_cols]
      .idxmax(axis=1)
      .apply(lambda x: int(x.split('_')[1]))
)

# 4) 결과 확인
print(df_dist[['dominant_topic_excl']].head())

   dominant_topic_excl
0                    3
1                    4
2                    6
3                    1
4                    1


In [35]:
print(df_dist.value_counts(subset=['dominant_topic_excl'],sort=False))

dominant_topic_excl
0                      688
1                      197
2                      325
3                      649
4                      283
5                      161
6                      606
7                      300
Name: count, dtype: int64


In [36]:
df_dist.head()

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,dominant_topic_excl
0,0.157803,0.001509,0.001509,0.553391,0.001509,0.001509,0.281260,0.001510,3
1,0.001204,0.001205,0.216162,0.001204,0.713332,0.001204,0.064485,0.001204,4
2,0.150836,0.198215,0.051539,0.001118,0.080849,0.001119,0.402748,0.113576,6
3,0.390330,0.508155,0.013061,0.084850,0.000901,0.000902,0.000901,0.000901,1
4,0.064038,0.930137,0.000971,0.000971,0.000971,0.000971,0.000971,0.000971,1


In [38]:
import pandas as pd
import numpy as np
import re

# === 파일 경로 ===
path_orig = "data/03_academia_2023_2025_with_methods.csv"                      # 원본
path_dist = "data/03_academia_lda_document_topic_distribution.csv"  # LDA 분포
out_path  = "data/topic_academia/academia_with_dominant_topic.csv"

# === 로드 ===
orig = pd.read_csv(path_orig, low_memory=False)
dist = pd.read_csv(path_dist, low_memory=False)

# (옵션) content가 content_text로 되어 있으면 맞춰주기
if "content" not in orig.columns and "content_text" in orig.columns:
    orig = orig.rename(columns={"content_text": "content"})

# 1) 원본에 dominant_topic이 이미 있으면 그대로 사용
if "dominant_topic" not in orig.columns:
    # 2) 없으면 분포파일에서 가져오거나 계산
    if "dominant_topic_excl" in dist.columns:
        dom_series = dist["dominant_topic_excl"]
    else:
        # Topic_0, Topic_1 ... 형태에서 argmax로 dominant topic 계산
        topic_cols = [c for c in dist.columns if re.match(r"^Topic_\d+$", c)]
        if not topic_cols:
            raise ValueError("분포 파일에 Topic_0, Topic_1 ... 형식의 컬럼이 없습니다.")
        topic_arr = dist[topic_cols].to_numpy()
        best_idx = topic_arr.argmax(axis=1)  # 각 행에서 최고 확률의 컬럼 인덱스
        topic_nums = np.array([int(re.search(r"\d+", c).group()) for c in topic_cols])
        dom_series = pd.Series(topic_nums[best_idx], index=dist.index, name="dominant_topic")

    # 행 순서 기준으로 dominant_topic만 원본에 붙이기
    orig["__rowid__"] = range(len(orig))
    dist["__rowid__"] = range(len(dist))
    tmp = pd.DataFrame({"__rowid__": dist["__rowid__"], "dominant_topic": dom_series})
    orig = orig.merge(tmp, on="__rowid__", how="left").drop(columns="__rowid__")

# 3) 최종 필요한 컬럼만 남기기
wanted = ["title", "abstract", "affiliations", "date","keywords", "dominant_topic"]
existing = [c for c in wanted if c in orig.columns]
missing  = [c for c in wanted if c not in orig.columns]

final = orig[existing].copy()
final.to_csv(out_path, index=False, encoding="utf-8-sig")

print(f"저장 완료: {out_path}  (rows={len(final)}, cols={len(final.columns)})")
if missing:
    print("다음 컬럼은 데이터에 없어 제외되었습니다:", missing)

저장 완료: data/topic_academia/academia_with_dominant_topic.csv  (rows=3209, cols=6)


In [39]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3209 entries, 0 to 3208
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           3209 non-null   object
 1   abstract        3209 non-null   object
 2   affiliations    3209 non-null   object
 3   date            3209 non-null   int64 
 4   keywords        3209 non-null   object
 5   dominant_topic  3209 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 150.6+ KB


In [40]:
import os
import re
import pandas as pd

# === 입력: 바로 'final'이 메모리에 없으면 직전 결과 파일에서 읽음 ===
if 'final' not in globals():
    final = pd.read_csv("data/topic_academia/merged_min_columns.csv", low_memory=False)

# 사용 컬럼 확인 (필수: dominant_topic)
required = {"title", "abstract", "affiliations","date", "keywords", "dominant_topic"}
missing = [c for c in required if c not in final.columns]
if missing:
    raise ValueError(f"다음 컬럼이 없습니다: {missing}")

# 저장 경로 (이미 존재한다고 하셨으므로 생성 생략해도 되지만, 안전하게 유지)
out_dir = "data/topic_academia"  # ← 요청 경로
os.makedirs(out_dir, exist_ok=True)

# dominant_topic이 있는 행만
df_topics = final.dropna(subset=['dominant_topic']).copy()

# 토픽 값 정리(숫자면 int로, 아니면 문자열 유지)
def normalize_topic(x):
    try:
        return int(float(x))
    except:
        return str(x)

df_topics['__topic_norm__'] = df_topics['dominant_topic'].apply(normalize_topic)

# 파일명 안전하게 만드는 함수
def safe_name(s):
    return re.sub(r'[^0-9A-Za-z._-]+', '_', str(s))

# 토픽별 저장
for t, g in df_topics.groupby('__topic_norm__', sort=True):
    fname = f"dominant_topic_{safe_name(t)}.csv"
    out_path = os.path.join(out_dir, fname)
    g.drop(columns=['__topic_norm__']).to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"Saved: {out_path} (rows={len(g)})")

Saved: data/topic_academia/dominant_topic_0.csv (rows=688)
Saved: data/topic_academia/dominant_topic_1.csv (rows=197)
Saved: data/topic_academia/dominant_topic_2.csv (rows=325)
Saved: data/topic_academia/dominant_topic_3.csv (rows=649)
Saved: data/topic_academia/dominant_topic_4.csv (rows=283)
Saved: data/topic_academia/dominant_topic_5.csv (rows=161)
Saved: data/topic_academia/dominant_topic_6.csv (rows=606)
Saved: data/topic_academia/dominant_topic_7.csv (rows=300)
